In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Scrape website for CPI (Consumer Price Index) to calculate for inflation to adjust revenue for 2019
res = requests.get("https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
CPI = pd.read_html(str(table))[0]
CPI.head()


,Year,Annual Average,Annual Percent Change (rate of inflation)
0,1913,9.9,NaN
1,1914,10.0,1.3%
2,1915,10.1,0.9%
3,1916,10.9,7.7%
4,1917,12.8,17.8%


In [26]:
# Clean data
CPI['Year'] = CPI['Year'].str.replace('*', '')
CPI.Year = CPI.Year.astype('int64') 
CPI.dropna()


,Year,Annual Average,Annual Percent Change (rate of inflation)
1,1914,10.0,1.3%
2,1915,10.1,0.9%
3,1916,10.9,7.7%
4,1917,12.8,17.8%
5,1918,15.0,17.3%
...,...,...,...
102,2015,237.0,0.1%
103,2016,240.0,1.3%
104,2017,245.1,2.1%
105,2018,251.1,2.4%


In [7]:
# Save to CSV
CPI.to_csv('CPI.csv') 


In [27]:
# File to Load (Remember to Change These)
movie_data = "Resources/final_movie_metadata.csv"

# Read the Data
movie_df = pd.read_csv(movie_data)

# Display the data table for preview
movie_df.head(20)


,title,budget,popularity,release_date,revenue,runtime,vote_average,vote_count
0,The Godfather,6000000,41.109264,1972,245066411,175,8.5,6024
1,The Shawshank Redemption,25000000,51.645403,1994,28341469,142,8.5,8358
2,Fight Club,63000000,63.869599,1999,100853753,139,8.3,9678
3,Schindler's List,22000000,41.725123,1993,321365567,195,8.3,4436
4,One Flew Over the Cuckoo's Nest,3000000,35.529554,1975,108981275,133,8.3,3001
5,The Dark Knight,185000000,123.167259,2008,1004558444,152,8.3,12269
6,Whiplash,3300000,64.299990,2014,13092000,105,8.3,4376
7,Psycho,806948,36.826309,1960,32000000,109,8.3,2405
8,The Godfather: Part II,13000000,36.629307,1974,47542841,200,8.3,3418
9,Pulp Fiction,8000000,140.950236,1994,213928762,154,8.3,8670


In [28]:
# Merge CPI and movie dataframes to get CPI for the release year
merged = pd.merge(movie_df, CPI, how="left", left_on="release_date", right_on="Year")
merged.head()

,title,budget,popularity,release_date,revenue,runtime,vote_average,vote_count,Year,Annual Average,Annual Percent Change (rate of inflation)
0,The Godfather,6000000,41.109264,1972,245066411,175,8.5,6024,1972,41.8,3.3%
1,The Shawshank Redemption,25000000,51.645403,1994,28341469,142,8.5,8358,1994,148.2,2.6%
2,Fight Club,63000000,63.869599,1999,100853753,139,8.3,9678,1999,166.6,2.2%
3,Schindler's List,22000000,41.725123,1993,321365567,195,8.3,4436,1993,144.5,3.0%
4,One Flew Over the Cuckoo's Nest,3000000,35.529554,1975,108981275,133,8.3,3001,1975,53.8,9.1%


In [45]:
# Calculate for adjusted revenue using formula (Revenue ÷ Annual Average) x 255.7 (which is 2019's CPI). Source: https://www.phoenixunion.org/Page/16117
CPI = (merged['revenue'] / merged['Annual Average']) * 255.7
revenue_adjusted_movie_metadata = pd.DataFrame({'title': merged['title'],
                                                'budget': merged['budget'],
                                                'popularity': merged['popularity'],
                                                'release_date': merged['release_date'],
                                                'revenue': CPI,
                                                'runtime': merged['runtime'],
                                                'vote_average': merged['vote_average'],
                                                'vote_count': merged['vote_count']})

revenue_adjusted_movie_metadata["revenue"] = revenue_adjusted_movie_metadata["revenue"].astype(float).map("{:,.2f}".format)
revenue_adjusted_movie_metadata

,title,budget,popularity,release_date,revenue,runtime,vote_average,vote_count
0,The Godfather,6000000,41.109264,1972,"1,499,126,346.72",175,8.5,6024
1,The Shawshank Redemption,25000000,51.645403,1994,"48,899,552.11",142,8.5,8358
2,Fight Club,63000000,63.869599,1999,"154,791,744.55",139,8.3,9678
3,Schindler's List,22000000,41.725123,1993,"568,672,494.68",195,8.3,4436
4,One Flew Over the Cuckoo's Nest,3000000,35.529554,1975,"517,964,907.39",133,8.3,3001
...,...,...,...,...,...,...,...,...
1076,The Last Airbender,150000000,9.544657,2010,"373,412,184.37",103,4.7,1180
1077,Ghost Rider: Spirit of Vengeance,57000000,9.421301,2011,"169,652,635.28",95,4.7,1163
1078,Fantastic Four,120000000,17.246484,2015,"181,231,524.46",100,4.4,2322
1079,Batman & Robin,125000000,17.038824,1997,"379,498,823.02",125,4.2,1447


In [46]:
# Save to CSV
revenue_adjusted_movie_metadata.to_csv('movie_metadata_adjusted_revenue.csv') 